In [1]:
import cpp_soft_info

# Load Data

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from result_saver import SaverProvider
provider = SaverProvider()

In [4]:
import numpy as np
from Scratch import metadata_loader

DEVICE = 'ibm_torino'

md = metadata_loader(_extract=True, _drop_inutile=True)
# md = md[md["job_status"] == "JobStatus.DONE"]
# md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
max_distance = 10
md = md[md["distance"] == max_distance]
md = md[md["backend_name"]==DEVICE]


md = md[md["shots"]==10000]
md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,tags,meas_level,shots,tags_xp,rep_delay,sampled_state,...,job_status,execution_date,extra,optimization_level,code,distance,rounds,logical,layout,descr
2528,2023-12-02 11:16:20.667000+01:00,torino_repcode,ibm_torino,cnng7x70k0p00088v8vg,[RepCode],1.0,10000.0,NaN,NaN,NaN,...,JobStatus.DONE,2023-12-02 11:18:18.144738+01:00,NaN,NaN,RepetitionCodeCircuit,10.0,10,1,"_is_hex=True, sherbrooke",diff rounds v2
2527,2023-12-02 11:16:18.322000+01:00,torino_repcode,ibm_torino,cnng7wq24wx0008fagdg,[RepCode],1.0,10000.0,NaN,NaN,NaN,...,JobStatus.DONE,2023-12-02 11:16:48.914779+01:00,NaN,NaN,RepetitionCodeCircuit,10.0,10,0,"_is_hex=True, sherbrooke",diff rounds v2


In [5]:
memories = {}
jobs = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()
    jobs[mmr_name] = job_id

memory_str = "mmr_log_1"
memory = memories[memory_str][:100]
job_id = jobs[memory_str]
print(memory.shape)

(100, 100)


# Initialize the code

In [6]:
import stim
import pymatching

from soft_info import get_repcode_layout, get_repcode_IQ_map
from Scratch import create_or_load_kde_grid

d = max_distance
synd_rounds = max_distance
synd_rounds = 10

circuit = stim.Circuit.generated("repetition_code:memory",
                                 distance=d,
                                 rounds=synd_rounds,
                                 after_clifford_depolarization=0.1)

model = circuit.detector_error_model(decompose_errors=True)

# layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend(DEVICE), _is_hex=True)

##### TORINO ####
distance = max_distance
import pickle
with open('longest_path_torino.pkl', 'rb') as f:
    path = pickle.load(f)
bounded_path = path[:2 * distance - 1]
layout = bounded_path[1::2] + bounded_path[::2] 


qubit_mapping = get_repcode_IQ_map(layout, synd_rounds) #Hardcoded for repetition codes
# print(qubit_mapping)
# print(len(qubit_mapping))

In [7]:
other_date = None # if none then it will find the closest to the tobecalib_job date
# other_date = "2023-11-22T10:30:00" # "2023-11-22T" works too
grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, job_id, num_grid_points=5, 
                                                           num_std_dev=0.3, other_date=other_date)

# takes 10s

Specified job execution date: 2023-12-02 11:18:18.144738+01:00
Found jobs for backend ibm_torino with closest execution date 2023-12-02 09:12:35.042892+00:00.
Searching for ibm_torino and 23.12.02_09h11_5pts_0.3std


In [8]:
# get_counts
counts = cpp_soft_info.get_counts(memory, qubit_mapping, grid_dict, processed_scaler_dict, synd_rounds)

# Decode

In [9]:
matching = pymatching.Matching.from_detector_error_model(model)
# p_data = 6.869e-3 # mean sherbrooke noise
p_data = 1.15e-2
result = cpp_soft_info.decode_IQ_shots(matching._matching_graph, memory,
                                           synd_rounds, 1, True, qubit_mapping, grid_dict,
                                           processed_scaler_dict, p_data=p_data, p_mixed=p_data/1, #p_mixed=1e-80, for d=30
                                           common_measure=-1, _bimodal=True, merge_strategy = "replace", _detailed=True)
print("num_errors:", result.num_errors, "out of", len(memory), "shots")
 
# takes 5s

num_errors: 25 out of 100 shots


# Draw result

In [1]:
from soft_info import draw_matching_graph

draw_matching_graph(matching=None, d=d, T=synd_rounds, syndromes=result.error_details[0].detection_syndromes,
                    matched_edges=result.error_details[0].matched_edges, figsize=(20,20), edge_list=result.error_details[0].edges)

NameError: name 'd' is not defined

# Other decodings

In [12]:
matching = pymatching.Matching.from_detector_error_model(model)
num_errors = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, memory, 
                                           synd_rounds, qubit_mapping, grid_dict, 
                                           processed_scaler_dict)
print("num_errors:", num_errors, "out of", len(memory), "shots")

# takes 1s

TypeError: decode_IQ_shots_flat(): incompatible function arguments. The following argument types are supported:
    1. (matching: pm::UserGraph, not_scaled_IQ_data: numpy.ndarray[numpy.complex128[m, n]], synd_rounds: int, logical: int, _resets: bool, qubit_mapping: dict[int, int], kde_grid_dict: dict[int, cpp_soft_info.GridData], scaler_params_dict: dict[int, tuple[tuple[float, float], tuple[float, float]]], _detailed: bool = False) -> cpp_soft_info.DetailedDecodeResult

Invoked with: <pymatching._cpp_pymatching.MatchingGraph object at 0x2a61c4af0>, array([[ 2.01382292e+08+9.06661220e+07j, -7.96659030e+07-3.06433120e+07j,
        -1.81834893e+08+1.34221650e+07j, ...,
        -5.06856980e+07-6.18721900e+06j,  6.33442520e+07+7.03769970e+07j,
         8.28337210e+07+6.37128690e+07j],
       [-1.27219759e+08+1.02905142e+08j, -1.24680639e+08+1.69321330e+07j,
        -9.58126480e+07+7.00267970e+07j, ...,
        -8.56362710e+07+3.97657700e+07j, -1.07189304e+08+3.24693790e+07j,
        -6.872

In [ ]:
matching = pymatching.Matching.from_detector_error_model(model)

def weight_to_prob(weight):
    return 1/(1+np.exp(weight))

p_data = 6.869e-3 # mean sherbrooke ECR error
p_mixed = p_data/1 # Same as weighted
p_meas = 9.900e-3 # mean Readout assignment error

num_errors = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, memory, 
                                           synd_rounds, qubit_mapping, grid_dict, processed_scaler_dict,
                                           p_data, p_mixed, p_meas, common_measure=-1)
                                        
print("num_errors:", num_errors, "out of", len(memory), "shots")
         
# takes 1s